In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

## Read and save data

In [10]:
acq_time = 50 # change here
acq_time_steps = 5
output_filename = 'alldata5.csv'

falice = 'QKD_Lab_data/input-keys.alice'
fbob   = 'QKD_Lab_data/input-keys.bob'
fdecoy = 'QKD_Lab_data/input-keys.decoy'

read_f_alice = open(falice, "rb")
read_f_bob = open(fbob, "rb")
read_f_decoy = open(fdecoy, "rb")

encoding_big = {'0': 'H', '1': 'V', '2': 'D', '3': 'A'}
encoding_lit = {'0': 'H', '1': 'D', '2': 'V', '3': 'A'}
decoy_lit = {'0' : 'S', '2' : 'W'}

def to_states(byte_input, encoding):
    num = int.from_bytes(byte_input, byteorder='big', signed=False)
    states = [(num & ii) >> jj for (ii,jj) in zip([192,48,12,3],[6,4,2,0])]
    states = [encoding[str(states[i])] for i in range(len(states))]
    states.reverse()
    return states

def decode(f, encoding, num=acq_time):
    decoded = []
    count = 0
    while count < num:
        N = f.read(8)
        N = int.from_bytes(N, 'big')

        for _ in range(N):
            decoded += to_states(f.read(1), encoding)
        count += 1
    return decoded


def decode_N(f, encoding, block_sizes, num=acq_time):
    decoded = []
    count = 0
    while count < num:
        N = f.read(8)
        N = int.from_bytes(N, 'big')
        block_sizes.append(N)
#             pbar = tqdm(range(N))

        for _ in range(N):
#                 pbar.set_description(f'Processing block {count+1}')
            decoded += to_states(f.read(1), encoding)
        count += 1
    return decoded








#LOOP

for k in tqdm(np.arange(int(acq_time/acq_time_steps))):
    #print('---------- ITERATION %i of %i ----------' % (k+1, acq_time/acq_time_steps))

    #print('Reading Alice:')
    alice_states = decode(read_f_alice, encoding_lit, num=acq_time_steps)
    #print('Reading Bob:')
    bob_states   = decode(read_f_bob, encoding_lit, num=acq_time_steps)

    block_sizes = []
    #print('Reading decoy:')
    decoy_states = decode_N(read_f_decoy, decoy_lit, block_sizes, num=acq_time_steps)

    column = np.array([])
    for (i, bs) in enumerate(block_sizes):
        column = np.concatenate([column, np.repeat(i + 1 + k*acq_time_steps, bs)]) 

    dataset = pd.DataFrame(list(zip(alice_states, bob_states, decoy_states, column)), columns=['A', 'B', 'D', 'N'])
    dataset = dataset.astype({'N': 'int16'}) # cast to int
    print('a',len(dataset))

    #Sifting
    dataset['A_basis_Z'] = (dataset['A']=='H') | (dataset['A']=='V')
    dataset['B_basis_Z'] = (dataset['B']=='H') | (dataset['B']=='V')
    dataset = dataset[dataset['A_basis_Z'] == dataset['B_basis_Z']]
    print('b',len(dataset))

    dataset.to_csv(output_filename, mode='a', header=False, index=False)
    c_names = dataset.columns


read_f_alice.close()
read_f_bob.close()
read_f_decoy.close()

  0%|          | 0/10 [00:00<?, ?it/s]

a 86447
b 43401


 20%|██        | 2/10 [00:03<00:14,  1.82s/it]

a 85194
b 42782


 30%|███       | 3/10 [00:05<00:11,  1.64s/it]

...

a 82316
b 41381


 90%|█████████ | 9/10 [00:13<00:01,  1.33s/it]

a 83702
b 41954


100%|██████████| 10/10 [00:14<00:00,  1.44s/it]

a 78756
b 39747


In [11]:
acq_time = 50 # change here
acq_time_steps = 10
output_filename = 'alldata10.csv'

falice = 'QKD_Lab_data/input-keys.alice'
fbob   = 'QKD_Lab_data/input-keys.bob'
fdecoy = 'QKD_Lab_data/input-keys.decoy'

read_f_alice = open(falice, "rb")
read_f_bob = open(fbob, "rb")
read_f_decoy = open(fdecoy, "rb")

encoding_big = {'0': 'H', '1': 'V', '2': 'D', '3': 'A'}
encoding_lit = {'0': 'H', '1': 'D', '2': 'V', '3': 'A'}
decoy_lit = {'0' : 'S', '2' : 'W'}

def to_states(byte_input, encoding):
    num = int.from_bytes(byte_input, byteorder='big', signed=False)
    states = [(num & ii) >> jj for (ii,jj) in zip([192,48,12,3],[6,4,2,0])]
    states = [encoding[str(states[i])] for i in range(len(states))]
    states.reverse()
    return states

def decode(f, encoding, num=acq_time):
    decoded = []
    count = 0
    while count < num:
        N = f.read(8)
        N = int.from_bytes(N, 'big')

        for _ in range(N):
            decoded += to_states(f.read(1), encoding)
        count += 1
    return decoded


def decode_N(f, encoding, block_sizes, num=acq_time):
    decoded = []
    count = 0
    while count < num:
        N = f.read(8)
        N = int.from_bytes(N, 'big')
        block_sizes.append(N)
#             pbar = tqdm(range(N))

        for _ in range(N):
#                 pbar.set_description(f'Processing block {count+1}')
            decoded += to_states(f.read(1), encoding)
        count += 1
    return decoded








#LOOP

for k in tqdm(np.arange(int(acq_time/acq_time_steps))):
    #print('---------- ITERATION %i of %i ----------' % (k+1, acq_time/acq_time_steps))

    #print('Reading Alice:')
    alice_states = decode(read_f_alice, encoding_lit, num=acq_time_steps)
    #print('Reading Bob:')
    bob_states   = decode(read_f_bob, encoding_lit, num=acq_time_steps)

    block_sizes = []
    #print('Reading decoy:')
    decoy_states = decode_N(read_f_decoy, decoy_lit, block_sizes, num=acq_time_steps)

    column = np.array([])
    for (i, bs) in enumerate(block_sizes):
        column = np.concatenate([column, np.repeat(i + 1 + k*acq_time_steps, bs)]) 

    dataset = pd.DataFrame(list(zip(alice_states, bob_states, decoy_states, column)), columns=['A', 'B', 'D', 'N'])
    dataset = dataset.astype({'N': 'int16'}) # cast to int
    print('a',len(dataset))

    #Sifting
    dataset['A_basis_Z'] = (dataset['A']=='H') | (dataset['A']=='V')
    dataset['B_basis_Z'] = (dataset['B']=='H') | (dataset['B']=='V')
    dataset = dataset[dataset['A_basis_Z'] == dataset['B_basis_Z']]
    print('b',len(dataset))

    dataset.to_csv(output_filename, mode='a', header=False, index=False)
    c_names = dataset.columns


read_f_alice.close()
read_f_bob.close()
read_f_decoy.close()

  0%|          | 0/5 [00:00<?, ?it/s]

a 171641
b 86397


 20%|██        | 1/5 [00:02<00:11,  2.78s/it]

a 167011
b 84588


 40%|████      | 2/5 [00:05<00:08,  2.77s/it]

...

 60%|██████    | 3/5 [00:08<00:05,  2.71s/it]

a 165214
b 83551


 80%|████████  | 4/5 [00:10<00:02,  2.73s/it]

a 162458
b 81676


100%|██████████| 5/5 [00:14<00:00,  2.83s/it]


In [12]:
output_filename = 'alldata10.csv'
data10 = pd.read_csv(output_filename, header=None, names = c_names)
data10

,A,B,D,N,A_basis_Z,B_basis_Z
0,V,V,S,1,True,True
1,H,H,S,1,True,True
2,V,V,S,1,True,True
3,H,H,S,1,True,True
4,H,H,S,1,True,True
...,...,...,...,...,...,...
420606,H,H,S,50,True,True
420607,V,V,S,50,True,True
420608,H,H,S,50,True,True
420609,H,H,S,50,True,True


In [13]:
output_filename = 'alldata5.csv'
data5 = pd.read_csv(output_filename, header=None, names = c_names)
data5

,A,B,D,N,A_basis_Z,B_basis_Z
0,V,V,S,1,True,True
1,H,H,S,1,True,True
2,V,V,S,1,True,True
3,H,H,S,1,True,True
4,H,H,S,1,True,True
...,...,...,...,...,...,...
419775,V,V,S,50,True,True
419776,V,V,S,50,True,True
419777,V,V,S,50,True,True
419778,V,V,S,50,True,True


In [17]:
data10.loc[:419779,:] == data5.loc[:419779,:]

,A,B,D,N,A_basis_Z,B_basis_Z
0,True,True,True,True,True,True
1,True,True,True,True,True,True
2,True,True,True,True,True,True
3,True,True,True,True,True,True
4,True,True,True,True,True,True
...,...,...,...,...,...,...
419775,True,True,True,True,True,True
419776,False,False,True,True,True,True
419777,True,True,True,True,True,True
419778,True,True,True,True,True,True


In [19]:
data10.dtypes

A            object
B            object
D            object
N             int64
A_basis_Z      bool
B_basis_Z      bool
dtype: object

In [5]:
acq_time = 50 # change here
acq_time_steps = 10
output_filename = 'alldata5.csv'

falice = 'QKD_Lab_data/input-keys.alice'
fbob   = 'QKD_Lab_data/input-keys.bob'
fdecoy = 'QKD_Lab_data/input-keys.decoy'

read_f_alice = open(falice, "rb")
read_f_bob = open(fbob, "rb")
read_f_decoy = open(fdecoy, "rb")

encoding_big = {'0': 'H', '1': 'V', '2': 'D', '3': 'A'}
encoding_lit = {'0': 'H', '1': 'D', '2': 'V', '3': 'A'}
decoy_lit = {'0' : 'S', '2' : 'W'}

def to_states(byte_input, encoding):
    num = int.from_bytes(byte_input, byteorder='big', signed=False)
    states = [(num & ii) >> jj for (ii,jj) in zip([192,48,12,3],[6,4,2,0])]
    states = [encoding[str(states[i])] for i in range(len(states))]
    states.reverse()
    return states

def decode(f, encoding, num=acq_time_steps):
    decoded = []
    count = 0
    while count < num:
        N = f.read(8)
        N = int.from_bytes(N, 'big')

        for _ in range(N):
            decoded += to_states(f.read(1), encoding)
        count += 1
    return decoded


def decode_N(f, encoding, block_sizes, num=acq_time_steps):
    decoded = []
    count = 0
    while count < num:
        N = f.read(8)
        N = int.from_bytes(N, 'big')
        block_sizes.append(N)
#             pbar = tqdm(range(N))

        for _ in range(N):
#                 pbar.set_description(f'Processing block {count+1}')
            decoded += to_states(f.read(1), encoding)
        count += 1
    return decoded








#LOOP

for k in tqdm(np.arange(int(acq_time/acq_time_steps))):
    #print('---------- ITERATION %i of %i ----------' % (k+1, acq_time/acq_time_steps))

    #print('Reading Alice:')
    alice_states = decode(read_f_alice, encoding_big, num=acq_time_steps)
    #print('Reading Bob:')
    bob_states   = decode(read_f_bob, encoding_big, num=acq_time_steps)

    block_sizes = []
    #print('Reading decoy:')
    decoy_states = decode_N(read_f_decoy, decoy_lit, block_sizes, num=acq_time_steps)

    column = np.array([])
    for (i, bs) in enumerate(block_sizes):
        column = np.concatenate([column, np.repeat(i + 1 + k*acq_time_steps, bs)]) 

    dataset = pd.DataFrame(list(zip(alice_states, bob_states, decoy_states, column)), columns=['A', 'B', 'D', 'N'])
    dataset = dataset.astype({'N': 'int16'}) # cast to int
    print('a',len(dataset))

    #Sifting
    dataset['A_basis_Z'] = (dataset['A']=='H') | (dataset['A']=='V')
    dataset['B_basis_Z'] = (dataset['B']=='H') | (dataset['B']=='V')
    dataset = dataset[dataset['A_basis_Z'] == dataset['B_basis_Z']]
    print('b',len(dataset))

    dataset.to_csv(output_filename, mode='a', header=False, index=False)
    c_names = dataset.columns


read_f_alice.close()
read_f_bob.close()
read_f_decoy.close()

  0%|          | 0/5 [00:00<?, ?it/s]

a 171641
b 129842


 20%|██        | 1/5 [00:02<00:11,  2.79s/it]

a 167011
b 126628


 40%|████      | 2/5 [00:05<00:08,  2.82s/it]

...

 60%|██████    | 3/5 [00:09<00:06,  3.33s/it]

a 165214
b 125347


 80%|████████  | 4/5 [00:12<00:03,  3.18s/it]

a 162458
b 123051


100%|██████████| 5/5 [00:15<00:00,  3.00s/it]


In [19]:
data2.loc[419776,:]

A               V
B               V
D               S
N              50
A_basis_Z    True
B_basis_Z    True
Name: 419776, dtype: object

In [7]:

sum(dataset.A_basis_Z)

65093

In [8]:
dataset

,A,B,D,N,A_basis_Z,B_basis_Z
3,D,D,S,41,False,False
4,D,A,S,41,False,False
5,D,D,S,41,False,False
7,H,H,S,41,True,True
8,D,D,S,41,False,False
...,...,...,...,...,...,...
162451,H,V,S,50,True,True
162453,H,H,S,50,True,True
162454,H,H,S,50,True,True
162455,H,V,S,50,True,True


In [7]:
dataset['A_basis_Z'] = (dataset['A']=='H') | (dataset['A']=='V')
dataset['B_basis_Z'] = (dataset['B']=='H') | (dataset['B']=='V')
dataset['Sifting']   = dataset['A_basis_Z'] == dataset['B_basis_Z']

In [8]:
sifted_data = dataset[dataset['Sifting'] == True]
# sifted_data.drop('Sifting')

# save df to csv file
# sifted_data.to_csv('sifted.csv', index=False)

## Analysis

In [9]:
p_t_HV = .90
p_t_D  = .10

p_r_HV = .50
p_r_DA = .50

p_d_S  = .70
p_d_W  = .30

mu1 = 0.4699
mu2 = 0.1093

In [10]:
def count_Z(df):
    n_Z    = sum(df['B_basis_Z'])
    df_mZ  = df[(df['A_basis_Z']) & (df['B_basis_Z'])]
    m_Z    = len(df_mZ[df_mZ['A']!= df_mZ['B']])
    
    return n_Z, m_Z

def compute_QBER_Z(df):
    n_Z, m_Z = count_Z(df)
    QBER_Z = m_Z / n_Z

    return QBER_Z
    
def count_X(df):
    n_X    = len(df) - sum(df['B_basis_Z'])
    df_mX  = df[ ~ ((df['A_basis_Z']) | (df['B_basis_Z']))]
    m_X    = len(df_mX[df_mX['A'] != df_mX['B']])
    
    return n_X, m_X

def compute_QBER_X(df):
    n_X, m_X = count_X(df)
    QBER_X = m_X / n_X
                 
    return QBER_X

In [30]:
c_names

Index(['A', 'B', 'D', 'N', 'A_basis_Z', 'B_basis_Z', 'Sifting'], dtype='object')

In [11]:
compute_QBER_X(sifted_data)


0.011809600061985873

In [12]:
compute_QBER_Z(sifted_data)

0.015714597341040255

In [13]:
def delta(n,eps):
    return np.sqrt(n*np.log(1/eps)*0.5)

def tau(n):
    strong_term = (p_d_S * np.exp(-mu1) * mu1**n) / np.math.factorial(n)
    weak_term = (p_d_W * np.exp(-mu2) * mu2**n) / np.math.factorial(n)
    return strong_term + weak_term

def gamma(a,b,c,d):
    fact1 = ((c+d)*(1-b)*b) / (c*d*np.log(2))
    fact2 = ((c+d)*21**2) / ((c*d)*(1-b)*b*a**2)
    return np.sqrt(fact1 * np.log2(fact2))

In [14]:
eps_sec = 1e-9
eps_cor = 1e-15
eps1    = 1/19 * eps_sec #define once for all
eps2    = 1/19 * eps_sec

# A16 + A14
def esse_u_basis_0(n_Z, m_Z, m_Z_mu1, m_Z_mu2):
    suz01 = 2 * ( tau(0) * (np.exp(mu1)/p_d_S) * (m_Z_mu1 + delta(m_Z, eps2)) + delta(n_Z, eps1) )
    suz02 = 2 * ( tau(0) * (np.exp(mu2)/p_d_W) * (m_Z_mu2 + delta(m_Z, eps2)) + delta(n_Z, eps1) )
    suz03 = 2 * ( m_Z + delta(n_Z, eps1) )
    
    return min(suz01, suz02, suz03)

# A17
def esse_l_basis_1(n_Z_mu1, n_Z_mu2, n_Z, suz0):
    return (tau(1)*mu1)/(mu2*(mu1-mu2)) * ( (n_Z_mu2 - delta(n_Z, eps1))                     \
                                            - (mu2**2/mu1**2) * (n_Z_mu1 + delta(n_Z, eps1)) \
                                            - (mu1**2-mu2**2)/(mu1**2) * (suz0 / tau(0)) )

# A19
def esse_l_basis_0(n_Z, n_Z_mu1, n_Z_mu2):
    res = (tau(0)/(mu1 - mu2)) * (mu1*(n_Z_mu2 - delta(n_Z, eps1)) - mu2*(n_Z_mu1 + delta(n_Z, eps1)))
    return max(res, 0)

# A22
def vi_u_X_1(m_X, m_X_mu1, m_X_mu2):
    return (tau(1)/(mu1 - mu2)) * ((m_X_mu1 + delta(m_X, eps2)) - (m_X_mu2 - delta(m_X, eps2)))

# A23
def phi_u_Z(v_x, s_x, s_z):
    r = v_x / s_x
    return r + gamma(eps_sec, r, s_z, s_x)

def h_2(x):
    return - x * np.log2(x) - (1-x) * np.log2(1-x)

# EQ1 : SKL
def ell(s0, s1, phi, QBER_Z):
    f = 1.5 # from 1.3 to 1.8
    lambda_EC = f * h_2(QBER_Z)
    return s0 + s1*(1 - h_2(phi)) - lambda_EC - 6 * np.log2(19 / eps_sec) - np.log2(2 / eps_cor)

In [15]:
# compute m_Z_k
n_Z_S, m_Z_S = count_Z(sifted_data[sifted_data['D']=='S'])
n_Z_W, m_Z_W = count_Z(sifted_data[sifted_data['D']=='W'])
# compute m_X_k
n_X_S, m_X_S = count_X(sifted_data[sifted_data['D']=='S'])
n_X_W, m_X_W = count_X(sifted_data[sifted_data['D']=='W'])

# compute n_Z, m_Z, QBER_Z
n_Z, m_Z = count_Z(sifted_data)
QBER_Z = compute_QBER_Z(sifted_data)
# compute n_X, m_X, QBER_X
n_X, m_X = count_X(sifted_data)
QBER_X = compute_QBER_X(sifted_data)

In [16]:
params_suz0 = {
    'n_Z': n_Z,
    'm_Z': m_Z,
    'm_Z_mu1' : m_Z_S,
    'm_Z_mu2' : m_Z_W,
    
}

suz0 = esse_u_basis_0(**params_suz0)
suz0

120248.19660247519

In [17]:
s1 = esse_l_basis_1(n_Z_S, n_Z_W, n_Z, suz0)
s1

362186.2520788632

In [18]:
s0 = esse_l_basis_0(n_Z, n_Z_S, n_Z_W)
s0

0

In [19]:
v_x = vi_u_X_1(m_X, m_X_S, m_X_W)
v_x

4321.2812208848745

In [20]:
params_sux0 = {
    'n_Z': n_X,
    'm_Z': m_X,
    'm_Z_mu1' : m_X_S,
    'm_Z_mu2' : m_X_W,
}

sux0 = esse_u_basis_0(**params_sux0)
sux0

16115.345191812761

In [21]:
s_x = esse_l_basis_1(n_X_S, n_X_W, n_X, sux0) # here is computed for X not for Z as the name suggests
s_z = s1
phi = phi_u_Z(v_x, s_x, s_z)
phi

0.2510727010577694

In [22]:
ell(s0, s1, phi, QBER_Z)

67482.40926097764

In [24]:
s_x

19357.92637363394

In [23]:
# sifted_data = pd.read_csv('sifted.csv')

# sifted_data.head()